In [2]:
import numpy as np
import random
import copy
import pandas as pd
#import ObjFunc
#import pandas as pd
import time

class EVCSIndividual:

    '''
    individual of demand oriented method
    '''
    def __init__(self,data,ev_ct,ec_cp,ev_n):

        self.data=data  #候选数据
        self.evcs=np.zeros(ev_n)  #每个充电站的车数
        self.individual=[]
        self.fitness=[]
        self.ev_ct=ev_ct
        self.ev_cp=ec_cp
        self.ev_n=ev_n

       
    def GenerateIndividual(self):
        '''
        generate a random chromsome for genetic algorithm
        '''
        #print("yes")
        gene = list()    #问题的解，基因，种群中的个体：[0，...，city_size]
        for i in range(len(self.data)):
            ind=sorted(self.data[i],key=lambda x:[x[1]])
            copy_ind=copy.deepcopy(ind[0])
            copy_ind.append(i)
            gene.append(copy_ind)
        #random.shuffle(gene)
        gene=sorted(gene, key=(lambda x: [x[0],x[1]]))
        self.individual=gene

    def calculateFitness(self,cd_n=5):
        '''
        calculate the fitness of the indiviual
        '''
        #distance = 0.0
        ev_n=np.zeros((self.ev_n,cd_n))  #单位时间内每个电桩充电车数
        cd_wt=np.zeros((self.ev_n,cd_n)) #充电桩工作总时间
        cd_st=np.zeros((self.ev_n,cd_n)) #充电桩空闲时间
        st=0.   #电桩空闲总时间  
        qt=0.   #电车排队总时间 
        tt=0.   #充电总时间 到充电桩时间+排队时间+充电时间     
        cost=0. #充电总成本成本
        n=[]  
        x_n=0
        for i in range(len(self.individual)): #len(self.indiviual)
            #cost+=self.individual[i][1]*self.ev_ct[i]
            cost+=self.individual[i][1]+self.ev_cp[self.individual[i][0]]*self.ev_ct[self.individual[i][2]]
            k=int(self.evcs[self.individual[i][0]]%cd_n)
            #print(k)
            self.evcs[self.individual[i][0]]+=1
            if cd_wt[self.individual[i][0]][k]<60:
                ev_n[self.individual[i][0]][k]+=1
            if self.individual[i][1]<cd_wt[self.individual[i][0]][k]:
                tt+=cd_wt[self.individual[i][0]][k]+self.ev_ct[self.individual[i][0]]
                qt+=cd_wt[self.individual[i][0]][k]-self.individual[i][1]
                #print("第%d辆排队的EV，车辆编号：%d,具体信息：%s,桩号：%d,需要排队时间：%f" %(x_n,i,str(self.indiviual[i]),k,cd_wt[self.indiviual[i][0]][k]-self.indiviual[i][1]))
                cd_wt[self.individual[i][0]][k]=cd_wt[self.individual[i][0]][k]+self.ev_ct[self.individual[i][0]]
                x_n+=1
            else:
                tt+=self.individual[i][1]+self.ev_ct[self.individual[i][0]]
                st+=self.individual[i][1]-cd_wt[self.individual[i][0]][k]
                cd_st[self.individual[i][0]][k]+=self.individual[i][1]-cd_wt[self.individual[i][0]][k]
                cd_wt[self.individual[i][0]][k]=self.individual[i][1]+self.ev_ct[self.individual[i][0]]
        revnue=0.
        t_ev_n=0
        t_st_v=0
        for i in range(self.ev_n):
            for j in range(cd_n):
                revnue+=(cd_wt[i][j]-cd_st[i][j])*self.ev_cp[i]
                t_ev_n+=ev_n[i][j]
                t_st_v+=cd_st[i][j]
        
        n.append(revnue)      #总收益  
        n.append(cost)        #总成本
        n.append(tt)          #总时间
        n.append(qt)          #总排队时间
        n.append(st)          #总空闲时间
        n.append(t_ev_n)      #单位时间每个充电桩充电电车总数
       # n.append(t_st_v)
        self.fitness=np.array(n)
def main():

    
    data = pd.read_csv("chargingstations.csv", delimiter=";", header=None).values  #读取34省市数据，以';'分割，dataframe
    cities = data[:, 1:]# 0行至最后一行，1列至最后一列
    cities_name = data[:, 0] #0行至最后一行，第0列
    city_size = data.shape[0]# shape属性返回包含行和列的元组
    locations = np.arange(cities.shape[0])    #问题的解，基因，种群中的个体：[0，...，city_size     
    '''读取电车数据'''
    ev_data_1000= pd.read_csv('ev_data_1000.csv').values 
    #print(ev_data_1000)
    ev_x=ev_data_1000[:, 1:2]
    ev_x=[j for i in ev_x for j in i]
    ev_y=ev_data_1000[:, 2:3]
    ev_y=[j for i in ev_y for j in i]
    ev_ld=ev_data_1000[:, 3:4]
    ev_ld=[j for i in ev_ld for j in i]
    ev_ct=ev_data_1000[:, 4:5]
    ev_ct=[j for i in ev_ct for j in i]
    ev_cp_34= pd.read_csv('ev_cp_34.csv').values 
    ev_cp=ev_cp_34[:,1:]
    ev_cp=[j for i in ev_cp for j in i]

    t_c_l=[] #每个电车的候选充电站
    for i in range(1000):
        c_l=[]
        for j in range(34):
            d=np.sqrt((ev_x[i] - cities[j][0]) ** 2 + (ev_y[i] - cities[j][1]) ** 2)
            #d=ct_distance(ev_x[i],ev_y[i],cities[j])
            if d<ev_ld[i]:
                c=[]
                c.append(j)      # 充电站序号
                c.append(d)      # 距离充电站距离，也是行驶时间
                #c.append(ev_cp[j])  #充电电价
                c_l.append(c)
        t_c_l.append(c_l)        

    for i in range(len(t_c_l)-1,-1,-1):
        if t_c_l[i]==[]:
            del ev_x[i]
            del ev_y[i]
            del ev_ld[i]
            del ev_ct[i]
            del t_c_l[i]
            
    ind=EVCSIndividual(t_c_l,ev_ct,ev_cp,len(cities))
    ind.GenerateIndividual()
    ind.calculateFitness()
    #print(ind.individual)
    print("True Fitness:",ind.fitness)
    print("Fitness:",0.3*(ind.fitness[3]/40000)+0.4*(ind.fitness[4]/3000)+0.3*(1-ind.fitness[5]/899))
    print("Average of queuing time (minute):",ind.fitness[3]/899)
    print("Average of idle time (minute):",ind.fitness[4]/170)
    print("Number of charged EV within an hour:",int(ind.fitness[5]))
    print("Total cost of all EV:",ind.fitness[1])
    print("Total revenue of EVCS:",ind.fitness[0])

if __name__ == '__main__':
    start=time.time()
    main()
    end=time.time()
    print("Time consuming:",end-start)

True Fitness: [17092.86447308 21235.90069034 87043.36816439 69387.99797678
   214.68698671   503.        ]
Fitness: 0.6811817461974539
Average of queuing time (minute): 77.18353501310136
Average of idle time (minute): 1.2628646276786906
Number of charged EV within an hour: 503
Total cost of all EV: 21235.900690337516
Total revenue of EVCS: 17092.8644730815
Time consuming: 0.15059685707092285
